## ConfigBox - Used for error handling


In [1]:
import yaml

yaml_file_path = "test.yaml"

# Open the YAML file and load its contents
with open(yaml_file_path, 'r') as yaml_file:
    data = yaml.safe_load(yaml_file)


print(data)
print(type(data))

{'name': 'Rajjo', 'email': 'relkarrajat@gmail.com'}
<class 'dict'>


In [5]:
data['name']

'Rajjo'

In [7]:
from box import ConfigBox

config_data = ConfigBox(data)

print(config_data.name)
print(config_data.email)

Rajjo
relkarrajat@gmail.com


In [10]:
type(config_data.name)

str

ensure_annotation


In [11]:
def get_product(x:int, y:int):
    return x*y

In [12]:
get_product(x=2, y="7")

'77'

In [13]:
from ensure import ensure_annotations

@ensure_annotations
def get_product(x:int, y:int):
    return x*y

In [15]:
# get_product(x=8, y='8')